In [36]:
from pyspark import SparkContext, SparkConf
import pandas as pd
import re
from operator import add
import numpy as np

In [ ]:
import os
data1 = pd.read_csv("_Answers.csv",encoding="latin-1", iterator = True, chunksize=100)
# print(data1)
# data1.apply(lambda row: row.replace("\n", " "))
# data1.to_csv("aaa.csv",encoding='utf-8')
# print(data1.iloc[0])
hasHeader = False
os.remove("aaa.csv")
for chunk in data1:
    #print(chunk)
    chunk["Body"] = chunk["Body"].map(lambda body: body.replace("\n", " "))
    chunk["Body"] = chunk["Body"].map(lambda body: body.replace(",", ";"))
    #chunk["Title"] = chunk["Title"].map(lambda body: body.replace(",", ";"))
    #print(chunk)
    #print(chunk["Body"])
    if hasHeader:
        chunk.to_csv("aaa.csv", mode = "a", encoding="utf-8", header=False, index = False)
    else:
        chunk.to_csv("aaa.csv", mode = "a", encoding="utf-8", header=True, index = False)

    #break
#     print(type(chunk))
#     for index, row in chunk.iterrows():
#         print(row)
#         print(row["Body"])
#         chunk.iloc[index]["Body"] = chunk.iloc[index]["Body"].replace("\n", " ")
#         print(chunk.iloc[index]["Body"])
#         break
# data1.to_csv("aaa.csv",encoding='utf-8')
# data1 = pd.read_csv("aaa.csv")
# data1.replace('\n','')
# data1.to_csv("bbb.csv")
print("finish")
df = spark.read.csv("aaa.csv",header=True);
print(df.take(3))

In [2]:
sc = SparkContext.getOrCreate()

In [3]:
questionFile=sc.textFile("processedQuestion.csv")
answerFile=sc.textFile("processedAnswer.csv")
tagFile = sc.textFile("_Tags.csv")
stopWordsFile = sc.textFile("stopwords.txt")
print("finished")

finished


In [4]:
questionWithHeader = questionFile.map(lambda line: line.split(",")).filter(lambda line: len(line)>1)
header = questionWithHeader.first() #extract header
question = questionWithHeader.filter(lambda x: x != header)
print(question.take(3))

[['80', '26.0', '2008-08-01T13:57:07Z', '', '26', 'SQLStatement.execute() - multiple queries in one statement', '"<p>I\'ve written a database generation script in <a href=""http://en.wikipedia.org/wiki/SQL"">SQL</a> and want to execute it in my <a href=""http://en.wikipedia.org/wiki/Adobe_Integrated_Runtime"">Adobe AIR</a> application:</p>  <pre><code>Create Table tRole (       roleID integer Primary Key       ;roleName varchar(40) ); Create Table tFile (     fileID integer Primary Key     ;fileName varchar(50)     ;fileDescription varchar(500)     ;thumbnailID integer     ;fileFormatID integer     ;categoryID integer     ;isFavorite boolean     ;dateAdded date     ;globalAccessCount integer     ;lastAccessTime date     ;downloadComplete boolean     ;isNew boolean     ;isSpotlight boolean     ;duration varchar(30) ); Create Table tCategory (     categoryID integer Primary Key     ;categoryName varchar(50)     ;parent_categoryID integer ); ... </code></pre>  <p>I execute this in Adobe A

In [5]:
stopWordsList = stopWordsFile.collect()


In [6]:
questionLower = question.map(lambda x: (x[0], (x[5]+x[6]).lower()))
print(questionLower.take(3))

[('80', 'sqlstatement.execute() - multiple queries in one statement"<p>i\'ve written a database generation script in <a href=""http://en.wikipedia.org/wiki/sql"">sql</a> and want to execute it in my <a href=""http://en.wikipedia.org/wiki/adobe_integrated_runtime"">adobe air</a> application:</p>  <pre><code>create table trole (       roleid integer primary key       ;rolename varchar(40) ); create table tfile (     fileid integer primary key     ;filename varchar(50)     ;filedescription varchar(500)     ;thumbnailid integer     ;fileformatid integer     ;categoryid integer     ;isfavorite boolean     ;dateadded date     ;globalaccesscount integer     ;lastaccesstime date     ;downloadcomplete boolean     ;isnew boolean     ;isspotlight boolean     ;duration varchar(30) ); create table tcategory (     categoryid integer primary key     ;categoryname varchar(50)     ;parent_categoryid integer ); ... </code></pre>  <p>i execute this in adobe air using the following methods:</p>  <pre><cod

In [7]:
def f(x): return re.sub('[^a-zA-Z]+', ' ', x)
questionLower = questionLower.mapValues(f)
print(questionLower.take(3))

[('80', 'sqlstatement execute multiple queries in one statement p i ve written a database generation script in a href http en wikipedia org wiki sql sql a and want to execute it in my a href http en wikipedia org wiki adobe integrated runtime adobe air a application p pre code create table trole roleid integer primary key rolename varchar create table tfile fileid integer primary key filename varchar filedescription varchar thumbnailid integer fileformatid integer categoryid integer isfavorite boolean dateadded date globalaccesscount integer lastaccesstime date downloadcomplete boolean isnew boolean isspotlight boolean duration varchar create table tcategory categoryid integer primary key categoryname varchar parent categoryid integer code pre p i execute this in adobe air using the following methods p pre code public static function runsqlfromfile filename string void var file file file applicationdirectory resolvepath filename var stream filestream new filestream stream open file fil

In [8]:
def f2(x): return x.split(" ")
questionPairRaw = questionLower.flatMapValues(f2)
print(questionPairRaw.take(3))

[('80', 'sqlstatement'), ('80', 'execute'), ('80', 'multiple')]


In [9]:
questionPair=questionPairRaw.filter(lambda x: x[1] not in stopWordsList)
print(questionPair.take(3))

[('80', 'sqlstatement'), ('80', 'multiple'), ('80', 'queries')]


In [25]:
questionPairfilter=questionPair.filter(lambda x:x[1]!="")
print(questionPairfilter.take(5))

[('80', 'sqlstatement'), ('80', 'multiple'), ('80', 'queries'), ('80', 'statement'), ('80', 've')]


In [26]:
questionPairStage1 = questionPairfilter.map(lambda x:(x[0]+"@"+x[1],1)).reduceByKey(add)
print(questionPairStage1.take(3))

[('80@statement', 2), ('260@scripting', 1), ('810@legacy', 1)]


In [30]:
print(questionPairStage1.take(5))

[('80@statement', 2), ('260@scripting', 1), ('810@legacy', 1), ('1010@wepdwujodu', 2), ('1070@', 1)]


In [27]:
N = questionPairStage1.count()
print(N)

64469259


In [29]:
print(questionPairStage1.take(5))

[('80@statement', 2), ('260@scripting', 1), ('810@legacy', 1), ('1010@wepdwujodu', 2), ('1070@', 1)]


In [22]:
questionPairStage2MapPhase = questionPairStage1.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
print(questionPairStage2MapPhase.take(5))

[('statement', '80=2'), ('scripting', '260=1'), ('legacy', '810=1'), ('wepdwujodu', '1010=2'), ('', '1070=1')]


In [33]:
stage2Map = questionPairStage2MapPhase.countByKey()
print("finish")

finish


In [34]:
print(stage2Map.get('statement'))

32326


In [42]:
questionPairStage2 = questionPairStage2MapPhase.map(lambda x:(x[0]+"@"+x[1].split("=")[0],(1+np.log(int(x[1].split("=")[1])))*np.log(N/stage2Map.get(x[0]))))
print(questionPairStage2.take(5))

[('statement@80', 12.864654056714706), ('scripting@260', 9.952592008883006), ('legacy@810', 10.00613440500754), ('wepdwujodu@1010', 29.272062876568928), ('@1070', 3.9317379203466034)]


In [45]:
questionPairStage3MapPhase = questionPairStage2.map(lambda x:(x[0].split("@")[1],x[0].split("@")[0]+"="+str(x[1])))
print(questionPairStage3MapPhase.take(5))

[('80', 'statement=12.864654056714706'), ('260', 'scripting=9.952592008883006'), ('810', 'legacy=10.00613440500754'), ('1010', 'wepdwujodu=29.272062876568928'), ('1070', '=3.9317379203466034')]


In [53]:
def f3_1(a,b): return float(a)+float(b.split("=")[1])*float(b.split("=")[1])
def f3_2(a,b): return float(a)+float(b)
questionPairStage3AggByKey = questionPairStage3MapPhase.aggregateByKey(0.0,f3_1,f3_2)
print(questionPairStage3AggByKey.take(5))

[('191010', 5074.003365361348), ('226970', 2258.900620871435), ('1157260', 18997.994160399725), ('1695830', 15225.933929664128), ('2369410', 8488.077651803182)]


In [57]:
print(questionPairStage3AggByKey.lookup('90'))

[2922.887889356336]


In [61]:
questionPairStage3AggByKeysqr = questionPairStage3AggByKey.map(lambda x:(x[0],np.sqrt(x[1])))
print(questionPairStage3AggByKeysqr.lookup('90'))

[54.06373913591564]


In [64]:
stage3Map = questionPairStage3AggByKeysqr.collectAsMap()
print(stage3Map.get('90'))

54.06373913591564


In [67]:
questionPairStage3 = questionPairStage3MapPhase.map(lambda x:(x[0],x[1].split("=")[0]+"="+str(float(x[1].split("=")[1])/stage3Map.get(x[0]))))
print(questionPairStage3.take(5))

[('31380', 'reason=0.16248503914702925'), ('35700', 'usb=0.15255058391552892'), ('42620', 'normalised=0.16504589374702966'), ('43180', 'independent=0.036884688869146094'), ('44660', 'csv=0.12725009991386094')]


In [72]:
def f4_1(a,b):
    for item in a:
        if float(b.split("=")[1])>float(item.split("=")[1]):
            item = b
            break
    return a
def f4_2(a,b):
    for item1 in b:
        for item2 in a:
            if float(item1.split("=")[1])>float(item2.split("=")[1]):
                item2 = item1
                break
    return a
questionPairStage4 = questionPairStage3.aggregateByKey(["a=0","b=0","c=0","d=0","e=0"],f4_1,f4_2)
print("finish")

finish


In [78]:
outMap=questionPairStage4.collectAsMap()
print("finish")

KeyboardInterrupt: 

In [ ]:
print(1)